# KLUE BERT를 이용한 네이버 영화 리뷰 감성분석

런타임 유형을 **GPU**로 설정하세요.<br>
훈련에 시간이 오래 걸리니 집에서 수행해도 됩니다.

원본 링크 : https://github.com/ukairia777/tensorflow-nlp-tutorial (<a href=https://creativecommons.org/licenses/by-nc-sa/2.0/kr/>CC BY-NC-SA 2.0 KR</a>)<br>
Modified by uramoon@kw.ac.kr

KLUE BERT는 한국어에 대해 학습된 BERT 모델입니다.
KLUE는 Korean Language Understanding Evaluation의 약어로 한국어 이해능력을 평가하는 벤치마크이며, BERT는 Bidirectional Encoder Representations from Transfomerers의 약어로 구글에서 개발한 자연어처리 모델입니다.


![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQc4oeBEEOISwWxlXG2X7XViY-2GAmHea9kUQ&usqp=CAU)

본 노트북에서는 KLUE BERT를 불러와서 전이학습으로 네이버 영화 리뷰에 대해 감성분석을 수행합니다.<br>
KLUE BERT는 모두의 말뭉치, 나무위키, 뉴스기사, 청와대 국민청원 등 62GB의 데이터로 훈련한 모델입니다.

## 패키지 설치


In [1]:
# KLUE BERT를사용하기 위한 패키지 설치
!pip install transformers==4.37.2
!pip install tensorflow==2.15.0.post1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 23.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
import transformers
from transformers import BertTokenizer, TFBertModel

## 데이터 준비하기

In [3]:
# 훈련 데이터와 테스트 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x78d116e8de70>)

In [5]:
# TODO: DataFrame으로 읽기
train_data = pd.read_csv('ratings_train.txt', sep='\t')#TODO
test_data = pd.read_csv('ratings_test.txt', sep='\t')#TODO

In [6]:
# TODO: 데이터 크기 확인하기
print('훈련용 리뷰 개수 :', train_data.size )#TODO) # 훈련용 리뷰 개수 출력
print('테스트용 리뷰 개수 :',test_data.size)#TODO) # 테스트용 리뷰 개수 출력

훈련용 리뷰 개수 : 450000
테스트용 리뷰 개수 : 150000


In [11]:
# TODO: 훈련 데이터에서 처음 5개 출력하기
train_data[:5] # 상위 5개 출력

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [12]:
# TODO: 테스트 데이터에서 처음 5개 출력하기
test_data[:5] # 상위 5개 출력

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [13]:
# 훈련 데이터에서 NULL 값을 포함한 행 제거
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data = train_data.reset_index(drop=True) # 인덱스 재설정
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [14]:
# TODO: 테스트 데이터에서 NULL 값을 포함한 행 제거
test_data = test_data.dropna(how = 'any') #TODO: Null 값이 존재하는 행 제거
test_data = test_data.reset_index(drop=True) #TODO: 인덱스 재설정
print(test_data.isnull().values.any()) #TODO: Null 값이 존재하는지 확인

False


In [15]:
# TODO: NULL 제거 후 데이터 크기 확인하기
print('훈련용 리뷰 개수 :', train_data.size) # 훈련용 리뷰 개수 출력
print('테스트용 리뷰 개수 :',test_data.size) # 테스트용 리뷰 개수 출력

훈련용 리뷰 개수 : 449985
테스트용 리뷰 개수 : 149991


## 토큰화

지난 노트북과 달리 토크나이저를 직접 만들지 않고 이미 훈련이 완료된 모델에서 불러옵니다.

In [16]:
# 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [17]:
# 단어를 정수로 인코딩
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[2, 1160, 2259, 2369, 2369, 4311, 20657, 2259, 5501, 13132, 1415, 2259, 23713, 3]


In [18]:
# 문자열을 토큰화, ##은 해당 토큰이 어절의 시작이 아님을 알려줍니다.
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['보', '##는', '##내', '##내', '그대로', '들어맞', '##는', '예측', '카리스마', '없', '##는', '악역']


In [19]:
# 문자열을 인코딩 (인공신경망이 이해하는 형태로 변환) 후 디코딩하기 (사람이 이해하는 형태로 변환)
# [CLS]는 BERT에서 문서의 시작을 나타내는 토큰
# [SEP]는 문장의 끝을 나타내는 토큰
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

'[CLS] 보는내내 그대로 들어맞는 예측 카리스마 없는 악역 [SEP]'

In [20]:
# 토큰을 하나씩 출력하는 코드
for elem in tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"):
  print(tokenizer.decode(elem))

[ C L S ]
보
# # 는
# # 내
# # 내
그 대 로
들 어 맞
# # 는
예 측
카 리 스 마
없
# # 는
악 역
[ S E P ]


In [26]:
# TODO: 다음 문자열을 토큰화 하세요.
print(tokenizer.tokenize("전율을 일으키는 영화. 다시 보고싶은 영화"))

['전', '##율', '##을', '일으키', '##는', '영화', '.', '다시', '보고', '##싶', '##은', '영화']


In [27]:
# TODO: 다음 문자열을 정수로 인코딩 하세요.
print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))

[2, 1537, 2534, 2069, 6572, 2259, 3771, 18, 3690, 4530, 2585, 2073, 3771, 3]


각 리뷰에서 몇 개의 토큰을 사용할 것인지 설정합니다. 지난 노트북과 동일하게 하기 위해 36으로 설정합니다.

In [28]:
# 모든 리뷰 토큰 개수의 평균 + 3 * 표준편차 = 36
# 평균 + 2 * 표준편차를 사용하신 분들도 있습니다.
max_seq_len = 36

In [29]:
# 36개의 토큰으로 문장 인코딩하기, 길이가 짧기 때문에 남는 자리는 0으로 채워집니다. (padding)
encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=max_seq_len, padding='max_length', truncation=True)
print(encoded_result)
print('길이 :', len(encoded_result))

[2, 1537, 2534, 2069, 6572, 2259, 3771, 18, 3690, 4530, 2585, 2073, 3771, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길이 : 36


## 데이터셋 만들기

훈련 데이터셋 (X_train: 리뷰, y_train: 레이블 [0:부정, 1:긍정])과 테스트 데이터셋 (X_test, y_test)를 만듭니다.

In [30]:
# 훈련 데이터셋 만들기

X_train = []

for sentence in tqdm(train_data['document']): # document에서 리뷰 하나씩 가져오기, tqdm이 진행률을 보여줍니다.
  encoded_sentence = tokenizer.encode(sentence, max_length=max_seq_len, padding='max_length', truncation=True)
  X_train.append(encoded_sentence)

X_train = np.array(X_train, dtype=int)
y_train = np.array(train_data['label'])

100%|██████████| 149995/149995 [00:40<00:00, 3731.76it/s]


In [31]:
# TODO: 테스트 데이터셋 만들기

X_test = []

# TODO
for sentence in tqdm(test_data['document']): # document에서 리뷰 하나씩 가져오기, tqdm이 진행률을 보여줍니다.
  encoded_sentence = tokenizer.encode(sentence, max_length=max_seq_len, padding='max_length', truncation=True)
  X_test.append(encoded_sentence)

X_test = np.array(X_test, dtype=int) #TODO
y_test = np.array(test_data['label']) #TODO

100%|██████████| 49997/49997 [00:15<00:00, 3131.77it/s]


## 모델 생성

지금까지는 이전 층의 출력이 다음 층에 모두 연결되는 Sequential 모델만을 사용했습니다. 우리는 BERT가 내보내는 출력 중 일부만을 사용하기 때문에 Sequential 모델을 사용하지 않고, Functional API를 사용해 일반 함수와 같이 각 층의 입력과 출력을 지정하여 모델을 만들 것입니다.

In [32]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# 입력층
inputs = Input(shape=(max_seq_len,), dtype=tf.int32) # max_seq_len (36) 개의 토큰을 입력받아 inputs로 출력합니다.

# KLUE BERT 모델 불러오기
bert = TFBertModel.from_pretrained("klue/bert-base", from_pt=True)(inputs) # inputs을 입력으로 받아 bert로 출력합니다.

# 출력층
output = Dense(1, activation='sigmoid')(bert[1]) # bert의 출력 중 두 번째 원소를 입력받아 하나로 선형결합하여 sigmoid로 0 혹은 1을 output에 출력합니다.

# 모델 정의
model = Model(inputs=inputs, outputs=output) # 모델의 입력은 inputs, 출력은 output

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [33]:
# 시간이 오래 걸리니 epochs를 1로 설정하여 훈련합니다.
# GPU에서 한 epoch이 15분 정도 걸리니 집에서 수행해도 됩니다.
model.fit(X_train, y_train, epochs=1, batch_size=64, validation_split=0.2)

1875/1875 [==============================] - 1029s 526ms/step - loss: 0.3003 - accuracy: 0.8710 - val_loss: 0.2700 - val_accuracy: 0.8867


In [34]:
# 테스트 데이터셋으로 평가합니다.
results = model.evaluate(X_test, y_test, batch_size=1024)
print("test loss, test acc: ", results)

49/49 [==============================] - 123s 3s/step - loss: 0.2755 - accuracy: 0.8831
test loss, test acc:  [0.2755108177661896, 0.8830729722976685]




GRU 혹은 LSTM으로 모델을 직접 만든 경우 85%를 넘는 것이 쉽지 않았습니다. <br>


In [35]:
# TODO: model을 이용하여 주어진 문장을 분류하는 함수를 만들어 보세요.
# Hint: 위에서 수행한 내용을 참고하세요.

def sentiment_predict(new_sentence):
  encoded_input = tokenizer.encode(new_sentence, max_length=max_seq_len, padding='max_length', truncation=True) #TODO
  encoded_input = np.array([encoded_input])
  score = model.predict(encoded_input)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [36]:
sentiment_predict('보던거라 계속보고있는데 전개도 느리고 주인공인 은희는 한두컷 나오면서 소극적인모습에 ')

1/1 [==============================] - 3s 3s/step
90.72% 확률로 부정 리뷰입니다.



In [37]:
sentiment_predict("스토리는 확실히 실망이였지만 배우들 연기력이 대박이였다 특히 이제훈 연기 정말 ... 이 배우들로 이렇게밖에 만들지 못한 영화는 아쉽지만 배우들 연기력과 사운드는 정말 빛났던 영화. 기대하고 극장에서 보면 많이 실망했겠지만 평점보고 기대없이 집에서 편하게 보면 괜찮아요. 이제훈님 연기력은 최고인 것 같습니다")

1/1 [==============================] - 0s 44ms/step
89.65% 확률로 긍정 리뷰입니다.



In [38]:
sentiment_predict("남친이 이 영화를 보고 헤어지자고한 영화. 자유롭게 살고 싶다고 한다. 내가 무슨 나비를 잡은 덫마냥 나에겐 다시 보고싶지 않은 영화.")

1/1 [==============================] - 0s 45ms/step
81.74% 확률로 부정 리뷰입니다.



In [39]:
sentiment_predict("이 영화 존잼입니다 대박")

1/1 [==============================] - 0s 46ms/step
98.39% 확률로 긍정 리뷰입니다.



In [40]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

1/1 [==============================] - 0s 43ms/step
98.03% 확률로 긍정 리뷰입니다.



In [41]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

1/1 [==============================] - 0s 43ms/step
56.63% 확률로 긍정 리뷰입니다.



In [42]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

1/1 [==============================] - 0s 45ms/step
96.98% 확률로 부정 리뷰입니다.



In [43]:
sentiment_predict('감독 뭐하는 놈이냐?')

1/1 [==============================] - 0s 43ms/step
99.39% 확률로 부정 리뷰입니다.



In [44]:
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')

1/1 [==============================] - 0s 125ms/step
96.74% 확률로 긍정 리뷰입니다.

